### Building & Testing Models with Sample Dataset

### Setup workspace

In [1]:
# import required packages
import pandas as pd
import numpy as np
import sqlite3
import pymysql
from datetime import datetime
import time
import pickle
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.tree import export_graphviz
import scipy.stats as ss
# ignore warnings
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error
import sklearn
import math
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsRegressor
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
import pickle
from sklearn.metrics import mean_absolute_error

### Functions to clean the dfs

In [2]:
# Function to clean the Leavetimes df
# Based off of the CleaningNotebook cleaning methods
def clean_leavetimes(x):
    
    categorical_columns = x[["TRIPID","PROGRNUMBER","STOPPOINTID","PLANNEDTIME_ARR", 'PLANNEDTIME_DEP', 'ACTUALTIME_ARR','ACTUALTIME_DEP','VEHICLEID']].columns
    # Convert data type to category for these columns
    for column in categorical_columns:
        x[column] = x[column].astype('category')
    
    # Converting dayofservice to datetime64
    x['DAYOFSERVICE'] = pd.to_datetime(x['DAYOFSERVICE'], infer_datetime_format=True)

    # Converting lastupdate to datetime64
    x['LASTUPDATE'] = pd.to_datetime(x['LASTUPDATE'], infer_datetime_format=True)
    
    return x

In [3]:
# Function to clean the trips df
# Based off of the CleaningNotebook cleaning methods
def clean_trips(x):
    categorical_columns = x[["TRIPID","LINEID", "ROUTEID", "DIRECTION", "PLANNEDTIME_ARR","PLANNEDTIME_DEP", "ACTUALTIME_ARR", "ACTUALTIME_DEP", "NOTE","JOURNEY_TIME"]].columns
    # Convert data type to category for these columns
    for column in categorical_columns:
        x[column] = x[column].astype('category')
    
    # Converting dayofservice to datetime64
    x['DAYOFSERVICE'] = pd.to_datetime(x['DAYOFSERVICE'], infer_datetime_format=True)

    # Converting lastupdate to datetime64
    x['LASTUPDATE'] = pd.to_datetime(x['LASTUPDATE'], infer_datetime_format=True)
    
    
    return x

In [4]:
# Function to clean the weather df
# Based off of the CleaningNotebook cleaning methods

def clean_weather(x):
    # Converting date to datetime
    x['date'] = pd.to_datetime(x['date'], infer_datetime_format=True)
    
    return x

### 2.0 Reading in Leavetimes Dataset

In [5]:
# Reading in the cleaned Leavetimes csv
df_leavetimes = pd.read_csv(r'C:\Users\jason\OneDrive - University College Dublin\Documents\MSc Computer Science\Summer Semester\Data\Notebooks\CleanedCSVs\rt_leavetimes_DB_2018_cleaned_sample.csv', index_col=[0])

In [6]:
df_leavetimes.head(5)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LASTUPDATE
0,2018-01-01 00:00:00,5972116,12,119,48030,48030,48012,48012,2693211,2018-01-08 17:21:10
1,2018-01-01 00:00:00,5966674,12,119,54001,54001,54023,54023,2693267,2018-01-08 17:21:10
2,2018-01-01 00:00:00,5959105,12,119,60001,60001,59955,59955,2693263,2018-01-08 17:21:10
3,2018-01-01 00:00:00,5966888,12,119,58801,58801,58771,58771,2693284,2018-01-08 17:21:10
4,2018-01-01 00:00:00,5965960,12,119,56401,56401,56309,56323,2693209,2018-01-08 17:21:10


In [7]:
# Pass the leavetimes df through the cleaning function
df_leavetimes = clean_leavetimes(df_leavetimes)
df_leavetimes.head(5)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LASTUPDATE
0,2018-01-01,5972116,12,119,48030,48030,48012,48012,2693211,2018-01-08 17:21:10
1,2018-01-01,5966674,12,119,54001,54001,54023,54023,2693267,2018-01-08 17:21:10
2,2018-01-01,5959105,12,119,60001,60001,59955,59955,2693263,2018-01-08 17:21:10
3,2018-01-01,5966888,12,119,58801,58801,58771,58771,2693284,2018-01-08 17:21:10
4,2018-01-01,5965960,12,119,56401,56401,56309,56323,2693209,2018-01-08 17:21:10


In [8]:
df_leavetimes.dtypes

DAYOFSERVICE       datetime64[ns]
TRIPID                   category
PROGRNUMBER              category
STOPPOINTID              category
PLANNEDTIME_ARR          category
PLANNEDTIME_DEP          category
ACTUALTIME_ARR           category
ACTUALTIME_DEP           category
VEHICLEID                category
LASTUPDATE         datetime64[ns]
dtype: object

### 3.0 Read in weather CSV

In [9]:
#read in cleaned weather csc
df_weather = pd.read_csv(r'C:\Users\jason\OneDrive - University College Dublin\Documents\MSc Computer Science\Summer Semester\Data\Notebooks\CleanedCSVs\weather2018_cleaned.csv', index_col=[0])

In [10]:
df_weather.head(5)

,date,rain,temp,msl
0,2018-01-01 00:00:00,0.0,4.6,991.0
1,2018-01-01 01:00:00,0.1,4.7,991.1
2,2018-01-01 02:00:00,0.0,4.8,991.1
3,2018-01-01 03:00:00,0.0,4.9,990.7
4,2018-01-01 04:00:00,0.0,5.3,990.3


In [11]:
df_weather.dtypes

date     object
rain     object
temp    float64
msl     float64
dtype: object

In [12]:
# Sending the weather df to the cleaning function
df_weather = clean_weather(df_weather)
df_weather.head(5)

,date,rain,temp,msl
0,2018-01-01 00:00:00,0.0,4.6,991.0
1,2018-01-01 01:00:00,0.1,4.7,991.1
2,2018-01-01 02:00:00,0.0,4.8,991.1
3,2018-01-01 03:00:00,0.0,4.9,990.7
4,2018-01-01 04:00:00,0.0,5.3,990.3


In [13]:
df_weather.dtypes

date    datetime64[ns]
rain            object
temp           float64
msl            float64
dtype: object

### Merge Weather and LeaveTimes Dataset

In [14]:
# Sorting in order to merge
df_leavetimes.sort_values('LASTUPDATE', inplace=True)
df_weather.sort_values('date', inplace=True)

In [15]:
# Merging 
merged_data = pd.merge_asof(df_leavetimes, df_weather, left_on="LASTUPDATE", right_on="date")

In [16]:
merged_data.head(5)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LASTUPDATE,date,rain,temp,msl
0,2018-01-01,5972116,12,119,48030,48030,48012,48012,2693211,2018-01-08 17:21:10,2018-01-08 17:00:00,0.0,3.7,1017.1
1,2018-01-01,5970997,7,4095,40517,40517,40414,40435,2693273,2018-01-08 17:21:10,2018-01-08 17:00:00,0.0,3.7,1017.1
2,2018-01-01,5971005,7,4095,59688,59688,59621,59636,2693273,2018-01-08 17:21:10,2018-01-08 17:00:00,0.0,3.7,1017.1
3,2018-01-01,5971001,8,4192,50137,50137,50117,50117,2693273,2018-01-08 17:21:10,2018-01-08 17:00:00,0.0,3.7,1017.1
4,2018-01-01,5970999,8,4192,45337,45337,45376,45376,2693273,2018-01-08 17:21:10,2018-01-08 17:00:00,0.0,3.7,1017.1


### 4.0 Read In Trips Data for merging

In [17]:
#read in cleaned trips data
df_trips = pd.read_csv(r'C:\Users\jason\OneDrive - University College Dublin\Documents\MSc Computer Science\Summer Semester\Data\Notebooks\CleanedCSVs\rt_trips_DB_2018_cleaned.csv', index_col=[0])

In [18]:
df_trips.shape

(1893367, 11)

In [19]:
df_trips.head(5)

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,LASTUPDATE,NOTE
0,1517961600000000000,6253783,68,68_80,1,87245,84600,87524.0,84600.0,1519819511000000000,",2967409,"
2,1517961600000000000,6254942,45A,45A_70,2,35512,32100,36329.0,32082.0,1519819511000000000,",2448968,"
3,1517961600000000000,6259460,25A,25A_273,1,57261,54420,58463.0,54443.0,1519819511000000000,",3094242,"
4,1517961600000000000,6253175,14,14_15,1,85383,81600,84682.0,81608.0,1519819511000000000,",2526331,"
5,1517961600000000000,6248240,77A,77A_28,2,41648,37200,42019.0,37538.0,1519819511000000000,",2966500,"


In [20]:
# df_trips = df_trips.drop(columns=["SUPPRESSED"])

In [21]:
# df_trips = df_trips.dropna()

In [22]:
df_trips.shape

(1893367, 11)

In [23]:
# Converting Actual time to int64 to subtract for new feature Journey Time. (Error on model when JOURNEY_TIME was left as a float)

df_trips['ACTUALTIME_ARR'] = df_trips['ACTUALTIME_ARR'].astype("int64")
df_trips['ACTUALTIME_DEP'] = df_trips['ACTUALTIME_DEP'].astype("int64")

In [24]:
# Creating an additional feature called Journey Time
df_trips['JOURNEY_TIME'] = df_trips['ACTUALTIME_ARR'] - df_trips['ACTUALTIME_DEP']

In [25]:
df_trips.dtypes

DAYOFSERVICE        int64
TRIPID              int64
LINEID             object
ROUTEID            object
DIRECTION           int64
PLANNEDTIME_ARR     int64
PLANNEDTIME_DEP     int64
ACTUALTIME_ARR      int64
ACTUALTIME_DEP      int64
LASTUPDATE          int64
NOTE               object
JOURNEY_TIME        int64
dtype: object

In [26]:
# Pass the leavetimes df through the cleaning function
df_trips = clean_trips(df_trips)
df_trips.head(5)

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,LASTUPDATE,NOTE,JOURNEY_TIME
0,2018-02-07,6253783,68,68_80,1,87245,84600,87524,84600,2018-02-28 12:05:11,",2967409,",2924
2,2018-02-07,6254942,45A,45A_70,2,35512,32100,36329,32082,2018-02-28 12:05:11,",2448968,",4247
3,2018-02-07,6259460,25A,25A_273,1,57261,54420,58463,54443,2018-02-28 12:05:11,",3094242,",4020
4,2018-02-07,6253175,14,14_15,1,85383,81600,84682,81608,2018-02-28 12:05:11,",2526331,",3074
5,2018-02-07,6248240,77A,77A_28,2,41648,37200,42019,37538,2018-02-28 12:05:11,",2966500,",4481


In [27]:
# df_trips = df_trips.replace(np.nan, '', regex=True)
# df_trips.head(5)

In [28]:
df_trips['MONTH'] = df_trips['DAYOFSERVICE'].dt.month
df_trips['DAY'] = df_trips['DAYOFSERVICE'].dt.dayofweek

In [29]:
df_trips.dtypes

DAYOFSERVICE       datetime64[ns]
TRIPID                   category
LINEID                   category
ROUTEID                  category
DIRECTION                category
PLANNEDTIME_ARR          category
PLANNEDTIME_DEP          category
ACTUALTIME_ARR           category
ACTUALTIME_DEP           category
LASTUPDATE         datetime64[ns]
NOTE                     category
JOURNEY_TIME             category
MONTH                       int64
DAY                         int64
dtype: object

In [30]:
df_trips.sort_values(by=['TRIPID'])

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,LASTUPDATE,NOTE,JOURNEY_TIME,MONTH,DAY
681660,2018-01-06,5955221,14,14_15,1,42553,37800,42078,37836,2018-01-15 08:25:25,",2513511,",4242,1,5
723197,2018-01-06,5955222,14,14_16,2,47867,43200,47707,43182,2018-01-15 08:25:25,",2513514,",4525,1,5
684398,2018-01-06,5955223,14,14_15,1,54105,49200,54128,49171,2018-01-15 08:25:25,",2513515,",4957,1,5
719581,2018-01-06,5955224,14,14_16,2,60094,55200,60025,55203,2018-01-15 08:25:25,",2429419,",4822,1,5
674864,2018-01-06,5955225,14,14_15,1,65751,61200,65332,61161,2018-01-15 08:25:25,",2513518,",4171,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28895,2018-12-30,8592206,67,67_6,2,61896,58500,63920,58522,2019-01-16 18:16:31,",2805863,",5398,12,6
458924,2018-12-23,8592206,67,67_6,2,61896,58500,62418,58521,2019-01-08 17:30:40,",2805863,",3897,12,6
438572,2018-12-26,8592207,67,67_4,1,67655,63900,68012,63877,2019-01-09 17:30:44,",2805865,",4135,12,2
442075,2018-12-23,8592207,67,67_4,1,67655,63900,68707,63915,2019-01-08 17:30:40,",2805865,",4792,12,6


### Practicing Modelling on route 68 direction 1

In [31]:
## Creating a df based off of lineid 68
route68 = (df_trips[df_trips['LINEID'] =='68'])
route68direction = (route68[route68['DIRECTION'] == 1])
route68direction.head(5)

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,LASTUPDATE,NOTE,JOURNEY_TIME,MONTH,DAY
0,2018-02-07,6253783,68,68_80,1,87245,84600,87524,84600,2018-02-28 12:05:11,",2967409,",2924,2,2
78,2018-02-18,6263183,68,68_80,1,43305,40200,43313,40528,2018-02-26 11:09:33,",2967850,",2785,2,6
570,2018-06-26,7111764,68,68_81,1,60472,55800,59962,55858,2018-07-04 12:21:00,",3221763,",4104,6,1
593,2018-06-21,7023711,68,68_78,1,57296,52200,56558,52307,2018-06-29 15:43:00,",3225498,",4251,6,3
638,2018-06-21,7023710,68,68_78,1,27106,23100,27124,23095,2018-06-29 15:43:00,",3223429,",4029,6,3


In [32]:
route68direction.shape

(5270, 14)

In [33]:
# Sorting route68 by LASTUPDATE
route68direction.sort_values('LASTUPDATE', inplace=True)

In [34]:
# Merging weather and trips
merged_data_trips = pd.merge_asof(route68direction, df_weather, left_on="LASTUPDATE", right_on="date")

In [35]:
merged_data_trips.head(5)

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,LASTUPDATE,NOTE,JOURNEY_TIME,MONTH,DAY,date,rain,temp,msl
0,2018-01-01,5969011,68,68_80,1,50690,47400,50073,47439,2018-01-08 17:21:10,",2967855,",2634,1,0,2018-01-08 17:00:00,0.0,3.7,1017.1
1,2018-01-01,5969016,68,68_79,1,80892,77400,80334,77446,2018-01-08 17:21:10,",2967982,",2888,1,0,2018-01-08 17:00:00,0.0,3.7,1017.1
2,2018-01-01,5969007,68,68_80,1,36106,33300,36465,33562,2018-01-08 17:21:10,",2967845,",2903,1,0,2018-01-08 17:00:00,0.0,3.7,1017.1
3,2018-01-01,5969012,68,68_80,1,62226,58800,61739,58796,2018-01-08 17:21:10,",2967859,",2943,1,0,2018-01-08 17:00:00,0.0,3.7,1017.1
4,2018-01-01,5969018,68,68_80,1,87356,84600,87048,84675,2018-01-08 17:21:10,",2967986,",2373,1,0,2018-01-08 17:00:00,0.0,3.7,1017.1


In [36]:
merged_data_trips.dtypes

DAYOFSERVICE       datetime64[ns]
TRIPID                   category
LINEID                   category
ROUTEID                  category
DIRECTION                category
PLANNEDTIME_ARR          category
PLANNEDTIME_DEP          category
ACTUALTIME_ARR           category
ACTUALTIME_DEP           category
LASTUPDATE         datetime64[ns]
NOTE                     category
JOURNEY_TIME             category
MONTH                       int64
DAY                         int64
date               datetime64[ns]
rain                       object
temp                      float64
msl                       float64
dtype: object

In [37]:
# % missing on new feature JOURNEY_TIME
journey_missing = 100 * (merged_data_trips['JOURNEY_TIME'].isna().sum()/merged_data_trips.shape[0])
journey_missing

0.0

### Prepare and split the dataset into two datasets: 70% training and 30% test for test models

In [38]:
# X for the input data to be predicted on
X = merged_data_trips.drop(columns = ["JOURNEY_TIME","DAYOFSERVICE", "LASTUPDATE", "date","NOTE", "DIRECTION","LINEID"])
# y for the out/goal set
y = merged_data_trips[["JOURNEY_TIME"]]

In [39]:
# Split the dataset into two datasets: 70% training and 30% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=20)

print("original range is: ", merged_data_trips.shape[0])
print("training range (70%):\t rows 0 to", round(X_train.shape[0]))
print("test range (30%): \t rows", round(X_train.shape[0]), "to", round(X_train.shape[0]) + X_test.shape[0])

original range is:  5270
training range (70%):	 rows 0 to 3689
test range (30%): 	 rows 3689 to 5270


### Building and Testing Models on Sample Route 68, Direction 1

Classification metrics can't handle a mix of multiclass and continuous targets, which means accuracy score is only for classification problems. For regression problems we can use: R2 Score, MSE (Mean Squared Error), RMSE (Root Mean Squared Error).

In [40]:
# X for the input data to be predicted on
X = merged_data_trips.drop(columns = ["JOURNEY_TIME","DAYOFSERVICE", "LASTUPDATE", "date","NOTE", "DIRECTION","LINEID"])
# y for the out/goal set
y = merged_data_trips[["JOURNEY_TIME"]]

In [41]:
# Split the dataset into two datasets: 70% training and 30% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=20)

print("original range is: ", merged_data_trips.shape[0])
print("training range (70%):\t rows 0 to", round(X_train.shape[0]))
print("test range (30%): \t rows", round(X_train.shape[0]), "to", round(X_train.shape[0]) + X_test.shape[0])

original range is:  5270
training range (70%):	 rows 0 to 3689
test range (30%): 	 rows 3689 to 5270


### Decision Tree 

In [42]:
# Generating r2 score for decision tree model
decisionTree = DecisionTreeRegressor()
decisionTree.fit(X_train, y_train)
predictions = decisionTree.predict(X_test)
decisionTreeR2 = r2_score(y_test, predictions)
decisionTreeR2

0.8846117762776994

In [43]:
# Generating mean absolute error score for decision tree model
decisionTreeMAE = mean_absolute_error(y_test, predictions)
decisionTreeMAE

139.59962049335863

In [44]:
# Calculating Root Mean Square Error
DecisionTreeMSE = sklearn.metrics.mean_squared_error(y_test, predictions)
                           
DecisionTreeRMSE = math.sqrt(DecisionTreeMSE)
DecisionTreeRMSE

290.8548622889792

### Random Forest Regressor

In [45]:
# Generating r2 score for random forest model
randForestModel = RandomForestRegressor(n_estimators = 10, random_state=10)
randForestModel.fit(X_train, y_train.values.ravel())
predictionTest = randForestModel.predict(X_test)
randForestR2 = r2_score(y_test, predictionTest)
randForestR2

0.902126361984501

In [46]:
# Generating mean absolute error score for Random Forest model
randForestMAE = mean_absolute_error(y_test, predictionTest)
randForestMAE

129.27906388361797

In [47]:
# Calculating Root Mean Square Error for Random Forest
RandforestMSE = sklearn.metrics.mean_squared_error(y_test, predictionTest)
                           
RandForestRMSE = math.sqrt(RandforestMSE)
RandForestRMSE

267.8726969731727

- Checking feature importances

In [48]:
# Checking for important features
feature_list = list(X.columns)
importance = pd.DataFrame({'feature': feature_list, 'importance':randForestModel.feature_importances_})
importance.sort_values('importance', ascending=False)

,feature,importance
4,ACTUALTIME_ARR,0.355522
5,ACTUALTIME_DEP,0.332794
7,DAY,0.194900
2,PLANNEDTIME_ARR,0.028971
1,ROUTEID,0.024147
0,TRIPID,0.023071
3,PLANNEDTIME_DEP,0.014946
9,temp,0.009299
10,msl,0.008587
6,MONTH,0.005674


### K-nearest Neighbour

In [49]:
# Generating r2 score for K-nearest Neighbour
knn_model = KNeighborsRegressor(n_neighbors = 10)
knn_model.fit(X_train, y_train.values.ravel())
knnTest = knn_model.predict(X_test)
knnR2 = r2_score(y_test, knnTest)
knnR2

0.33231718036654656

In [50]:
# Generating mean absolute error score for k-nearest neighbour model
knnMAE = mean_absolute_error(y_test, knnTest)
knnMAE

487.94667931688804

In [51]:
# Calculating Root Mean Square Error for Knn
KnnMSE = sklearn.metrics.mean_squared_error(y_test, knnTest)
                           
KnnRMSE = math.sqrt(KnnMSE)
KnnRMSE

699.6497529972195

### Linear Regression

In [52]:
# Generating r2 score for Linear Regression
linearModel = LinearRegression().fit(X_train, y_train.values.ravel())
linearModel
linear_prediction = linearModel.predict(X_test)
linearRegR2 = r2_score(y_test, linear_prediction)
linearRegR2

1.0

In [53]:
# Generating mean absolute error score for Linear Regression
linearRegMAE = mean_absolute_error(y_test, linear_prediction)
linearRegMAE

1.6860167910760195e-11

In [55]:
# Calculating Root Mean Square Error for Linear Regression
LinRegMSE = sklearn.metrics.mean_squared_error(y_test, linear_prediction)
                           
LinearRegRMSE = math.sqrt(LinRegMSE)
LinearRegRMSE

2.052894064672558e-11

In [56]:
# Print the weights learned for each feature.
print("\nFeatures are: \n", X_train.columns)
print("\nCoeficients are: \n", linearModel.coef_)
print("\nIntercept is: \n", linearModel.intercept_)
print("\nFeatures and coeficients: \n", list(zip(X_train.columns, linearModel.coef_)))


Features are: 
 Index(['TRIPID', 'ROUTEID', 'PLANNEDTIME_ARR', 'PLANNEDTIME_DEP',
       'ACTUALTIME_ARR', 'ACTUALTIME_DEP', 'MONTH', 'DAY', 'rain', 'temp',
       'msl'],
      dtype='object')

Coeficients are: 
 [ 1.30114194e-18  3.46789264e-12 -3.88578059e-16 -8.32667268e-16
  1.00000000e+00 -1.00000000e+00  7.35661532e-14  8.35538236e-14
  6.04883439e-14  7.75508130e-15 -6.73072709e-16]

Intercept is: 
 -2.3917891667224467e-08

Features and coeficients: 
 [('TRIPID', 1.3011419399986416e-18), ('ROUTEID', 3.467892639719139e-12), ('PLANNEDTIME_ARR', -3.885780586188048e-16), ('PLANNEDTIME_DEP', -8.326672684688674e-16), ('ACTUALTIME_ARR', 1.0000000000000004), ('ACTUALTIME_DEP', -0.9999999999999982), ('MONTH', 7.356615316922444e-14), ('DAY', 8.35538235821609e-14), ('rain', 6.048834393437841e-14), ('temp', 7.755081299354316e-15), ('msl', -6.730727086790012e-16)]


- Printing Actual Death vs predicted based on train data

In [57]:
# calculate the prediction and threshold the value.
multiple_linreg_predictions_train = (linearModel.predict(X_train)) 

print("\nPredictions with multiple linear regression: \n")
actual_vs_predicted_multiplelinreg = pd.concat([y_train, pd.DataFrame(multiple_linreg_predictions_train, columns=['Predicted'])], axis=1)
print(actual_vs_predicted_multiplelinreg.head(100))


Predictions with multiple linear regression: 

   JOURNEY_TIME  Predicted
0          2634     3832.0
1          2888     4219.0
2          2903     3120.0
3          2943     3604.0
4          2373     5543.0
..          ...        ...
95         2704     3303.0
96         3407     3399.0
97          NaN     4240.0
98         3563     3706.0
99         5161     3119.0

[100 rows x 2 columns]


- Printing Actual Death vs predicted based on test data

In [58]:
# calculate the prediction and threshold the value. If >= 0.5 its true
multiple_linreg_predictions_test = (linearModel.predict(X_test)) * 1.0

print("\nPredictions with multiple linear regression: \n")
actual_vs_predicted_multiplelinreg = pd.concat([y_test, pd.DataFrame(multiple_linreg_predictions_test, columns=['Predicted'])], axis=1)
print(actual_vs_predicted_multiplelinreg.head(100))


Predictions with multiple linear regression: 

   JOURNEY_TIME  Predicted
0           NaN     3398.0
1           NaN     3801.0
2           NaN     3779.0
3           NaN     3868.0
4           NaN     5201.0
..          ...        ...
95          NaN     4014.0
96          NaN     3203.0
97         3732     3577.0
98          NaN     4660.0
99          NaN     6597.0

[100 rows x 2 columns]


### Printing and Discussing Results

In [59]:
# Some more evaluation metrics.
print("==================== Predictive Model Accuracy - R2 Score =======================\n")
print("Decision Tree : ", decisionTreeR2)
print("Random Forest : ", randForestR2)
print("K-nearest Neighbour : ", knnR2)
print("Linear Regression : ", linearRegR2 )

print("\n==================== Predictive Model Accuracy - MAE Score =======================\n")
print("Decision Tree : ", decisionTreeMAE)
print("Random Forest : ", randForestMAE)
print("K-nearest Neighbour : ", knnMAE)
print("Linear Regression : ", linearRegMAE )

print("\n==================== Predictive Model Accuracy - RMSE Score =======================\n")
print("Decision Tree : ", DecisionTreeRMSE)
print("Random Forest : ", RandForestRMSE)
print("K-nearest Neighbour : ", KnnRMSE)
print("Linear Regression : ", LinearRegRMSE )

==================== Predictive Model Accuracy - R2 Score =======================

Decision Tree :  0.8846117762776994
Random Forest :  0.902126361984501
K-nearest Neighbour :  0.33231718036654656
Linear Regression :  1.0

==================== Predictive Model Accuracy - MAE Score =======================

Decision Tree :  139.59962049335863
Random Forest :  129.27906388361797
K-nearest Neighbour :  487.94667931688804
Linear Regression :  1.6860167910760195e-11

==================== Predictive Model Accuracy - RMSE Score =======================

Decision Tree :  290.8548622889792
Random Forest :  267.8726969731727
K-nearest Neighbour :  699.6497529972195
Linear Regression :  2.052894064672558e-11


#### Decision Tree
- We get very good results from R2 score, MAE score and the RMSE score. 
- The decision tree is giving us very good results, we will likely use this method but will continue to test other predictive models first.

#### Random Forest
- Random Forest produces slightly better results than decision tree for the R2 Score, MAE score and RMSE score, however the random forest takes up far more space and takes longer to run. The trade off between accuracy and time taken/storage is something we have to strongly consider, as we have very large data sets. 
- After testing the size of a decision tree vs a random forest, the random forest pickle file is approximately 6.4 times bigger than the size of a decision tree pickle file (455KB vs 2896KB)

#### Knn
- We have gotten poor results from the R2 score, MAE score and the RMSE score.
- Due to these poor results, we will disregard this method for our predictive models.

#### Linear Regression
- An R2=1 means that the data is perfectly correlated. However, R2=1 means that for some reason our model predicts the response variable perfectly, which is generally too good to be true. It is likely we have multicollinearity here, which is where two or more independent variables have a strong correlation, giving an overly optimistic result. 
- In relation to the RMSE score, it is impossible to achieve RMSE lower than 0.5 due to large range of numerical values for this dataset, further pointing to our overly optimistic results likely due to multicollinearity.
- Similarly with the results of the MAE score, these values are too unrealistic, further hinting at an issue with multicollinearity, we will disregard this model going forward.

### Conclusion
- Based on the results above and considering the trade off between accuracy and time taken/storage for the models, we believe <b>Decision Tree</b> is the best option and we will be proceeding with this predictive model for the rest of our dataset.